In [1]:
import pandas as pd
import numpy as np
import requests
import bokeh.plotting as bk
from bokeh.models import HoverTool, Label, BoxZoomTool, PanTool, ZoomInTool, ZoomOutTool, ResetTool
from bokeh.plotting import figure, output_file, show



In [2]:
res = requests.get(f'https://api.iextrading.com/1.0/stock/TSLA/chart/5y')

In [3]:
data = res.json()

In [4]:
df = pd.DataFrame(data)

In [5]:
df['date'] = pd.to_datetime(df['date'])
df.date

0      2013-12-20
1      2013-12-23
2      2013-12-24
3      2013-12-26
4      2013-12-27
5      2013-12-30
6      2013-12-31
7      2014-01-02
8      2014-01-03
9      2014-01-06
10     2014-01-07
11     2014-01-08
12     2014-01-09
13     2014-01-10
14     2014-01-13
15     2014-01-14
16     2014-01-15
17     2014-01-16
18     2014-01-17
19     2014-01-21
20     2014-01-22
21     2014-01-23
22     2014-01-24
23     2014-01-27
24     2014-01-28
25     2014-01-29
26     2014-01-30
27     2014-01-31
28     2014-02-03
29     2014-02-04
          ...    
1228   2018-11-06
1229   2018-11-07
1230   2018-11-08
1231   2018-11-09
1232   2018-11-12
1233   2018-11-13
1234   2018-11-14
1235   2018-11-15
1236   2018-11-16
1237   2018-11-19
1238   2018-11-20
1239   2018-11-21
1240   2018-11-23
1241   2018-11-26
1242   2018-11-27
1243   2018-11-28
1244   2018-11-29
1245   2018-11-30
1246   2018-12-03
1247   2018-12-04
1248   2018-12-06
1249   2018-12-07
1250   2018-12-10
1251   2018-12-11
1252   201

In [6]:
seqs = np.arange(df.shape[0])
df['seqs']= pd.Series(seqs)

In [7]:
df.sample(2)


,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap,seqs
980,0.00,1.115261,0.000,302.99,2017-11-10,308.36,"Nov 10, 17",301.85,302.50,4625429,4625429,304.2703,980
990,1.26,1.211743,0.399,316.81,2017-11-27,317.34,"Nov 27, 17",309.51,313.25,4555894,4555894,313.9314,990


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 13 columns):
change              1258 non-null float64
changeOverTime      1258 non-null float64
changePercent       1258 non-null float64
close               1258 non-null float64
date                1258 non-null datetime64[ns]
high                1258 non-null float64
label               1258 non-null object
low                 1258 non-null float64
open                1258 non-null float64
unadjustedVolume    1258 non-null int64
volume              1258 non-null int64
vwap                1258 non-null float64
seqs                1258 non-null int64
dtypes: datetime64[ns](1), float64(8), int64(3), object(1)
memory usage: 127.8+ KB


In [9]:
seqs = np.arange(df.shape[0])
df['seqs'] = pd.Series(seqs)

In [10]:
df['changePercent'] = df['changePercent'].apply(lambda x: str(x) + '%')

In [11]:
df['mid'] = df.apply(lambda x: (x['open'] + x['close']) / 2, axis=1)

In [12]:
df['height']= df.apply(
    lambda x: x['close'] - x['open'] if x['close'] != x['open'] else 0.01,
    axis = 1 
)

In [13]:
inc = df.close > df.open
dec = df.close < df.open
w = .3

In [14]:
sourceInc = bk.ColumnDataSource(df.loc[inc])
sourceDec = bk.ColumnDataSource(df.loc[dec])

In [15]:
hover = HoverTool(
    tooltips=[
        ('date', '@date'),
        ('low', '@low'),
        ('high', '@high'),
        ('open', '@open'),
        ('close', '@close'),
        ('percent', '@changePercent'),
    ]
)

In [16]:
TOOLS = [hover, BoxZoomTool(), PanTool(), ZoomInTool(), ZoomOutTool(), ResetTool()]

In [17]:
p = bk.figure(plot_width = 1500, plot_height=800,x_axis_type="datetime", tools=TOOLS, title='<<needs to be dynamic>>', toolbar_location='above')
# p = bk.figure(plot_width = 1500, plot_height=800, tools=TOOLS, title='<<needs to be dynamic>>', toolbar_location='above')

In [18]:
p.xaxis.major_label_orientation = np.pi/4
p.grid.grid_line_alpha=w

In [19]:
descriptor = Label(x=70, y=70, text='Come up with a label')
p.add_layout(descriptor)

In [20]:
p.segment(df.seqs[inc], df.high[inc], df.seqs[inc], df.low[inc], color= 'green')



GlyphRenderer(id='1043', ...)

In [21]:
p.segment(df.seqs[dec], df.high[dec], df.seqs[dec], df.low[dec], color= 'red')

GlyphRenderer(id='1050', ...)

In [22]:

p.rect(x='seqs', y='mid', width=w, height='height', fill_color='green', line_color='green', source=sourceInc)

GlyphRenderer(id='1057', ...)

In [23]:

p.rect(x='seqs', y='mid', width=w, height='height', fill_color='red', line_color='red', source=sourceDec)

GlyphRenderer(id='1064', ...)

In [24]:
bk.show(p)

In [32]:
bk.save(p, './src/templates/candlestick.html', title= "5yr_candlestick")


'/home/mthwbrwn/codefellows_401/stocks_app/src/templates/candlestick.html'

In [ ]:
# generate render in as template 